#### Importing and organizing data

In [1]:
import kagglehub
import shutil
import os

# Download the dataset (cached path)
cached_path = kagglehub.dataset_download("roudranildas/chicken-images-classification-dataset")

# Define target path (working directory)
target_path = "/content/"

# Copy from cache to working directory
shutil.copytree(cached_path, target_path, dirs_exist_ok=True)

# New structure path
base_dir = "/content/organized-data"
splits = ['train', 'val', 'test']
classes = ['chicken', 'duck']

# Create new folders
for split in splits:

    for cls in classes:
        os.makedirs(os.path.join(base_dir, split, cls), exist_ok=True)

# Copy files
for cls in classes:
    src_root = f"/content/data/{cls}-images/data"
    for split in splits:
        src = os.path.join(src_root, split)
        dst = os.path.join(base_dir, split, cls)
        for file in os.listdir(src):
            shutil.copy(os.path.join(src, file), dst)


100%|██████████| 43.0M/43.0M [00:00<00:00, 155MB/s]

Extracting files...


# Step 1:  Import necessary libraries

In [2]:
# Step 1: Setup
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


# Step 2: Load dataset

In [3]:
from torchvision import datasets, transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

base_dir = "/content/organized-data"

train_ds = datasets.ImageFolder(root=os.path.join(base_dir, 'train'), transform=transform)
val_ds = datasets.ImageFolder(root=os.path.join(base_dir, 'val'), transform=transform)
test_ds = datasets.ImageFolder(root=os.path.join(base_dir, 'test'), transform=transform)

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=16, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=16, shuffle=False)

print("Classes:", train_ds.classes)

Classes: ['chicken', 'duck']


# Step 3: Load pre-trained model

In [4]:
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # Chicken vs Duck
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 83.5MB/s]


In [25]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [5]:
def evaluate(model, val_loader):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = val_loss / len(val_loader)
    accuracy = 100 * correct / total
    return avg_loss, accuracy


# Step 4: Training

In [9]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward + Backward + Optimize
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validation step
    val_loss, val_acc = evaluate(model, val_loader)

    print(f"Epoch [{epoch+1}/{num_epochs}]")
    print(f"Train Loss: {running_loss/len(train_loader):.4f} | "
          f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%\n")


Epoch [1/20]
Train Loss: 0.0151 | Val Loss: 0.1893 | Val Acc: 94.41%

Epoch [2/20]
Train Loss: 0.0456 | Val Loss: 0.2296 | Val Acc: 93.17%

Epoch [3/20]
Train Loss: 0.0145 | Val Loss: 0.2399 | Val Acc: 90.68%

Epoch [4/20]
Train Loss: 0.0201 | Val Loss: 0.1860 | Val Acc: 91.30%

Epoch [5/20]
Train Loss: 0.0290 | Val Loss: 0.2033 | Val Acc: 92.55%

Epoch [6/20]
Train Loss: 0.0125 | Val Loss: 0.2014 | Val Acc: 93.17%

Epoch [7/20]
Train Loss: 0.0606 | Val Loss: 0.2856 | Val Acc: 89.44%

Epoch [8/20]
Train Loss: 0.0465 | Val Loss: 0.2440 | Val Acc: 91.93%

Epoch [9/20]
Train Loss: 0.0156 | Val Loss: 0.2658 | Val Acc: 91.93%

Epoch [10/20]
Train Loss: 0.0232 | Val Loss: 0.2131 | Val Acc: 92.55%

Epoch [11/20]
Train Loss: 0.0288 | Val Loss: 0.3006 | Val Acc: 91.93%

Epoch [12/20]
Train Loss: 0.0101 | Val Loss: 0.1740 | Val Acc: 93.79%

Epoch [13/20]
Train Loss: 0.1200 | Val Loss: 0.4225 | Val Acc: 92.55%

Epoch [14/20]
Train Loss: 0.0253 | Val Loss: 0.2719 | Val Acc: 92.55%

Epoch [15/20]
T

# Step 6: Evaluate the Model on Test Data

In [20]:
def generate_classification_report(model, dataloader, class_names, device, name=""):
    model.eval()
    preds_list = []
    labels_list = []

    with torch.no_grad():
        for batch in dataloader:
            if isinstance(batch, (list, tuple)):
                images, labels = batch
            else:
                images = batch
                labels = None  # Just in case, unlikely in this use case

            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, preds = torch.max(outputs, 1)

            preds_list.extend(preds.cpu().tolist())
            labels_list.extend(labels.cpu().tolist())

    print(f"\n📊 Classification Report - {name} Set\n")
    print(classification_report(labels_list, preds_list, target_names=class_names))


In [22]:
class_names = train_ds.classes
generate_classification_report(model, test_loader, class_names, device, name="Test")


📊 Classification Report - Test Set

              precision    recall  f1-score   support

     chicken       0.99      0.83      0.90       172
        duck       0.91      0.99      0.95       310

    accuracy                           0.93       482
   macro avg       0.95      0.91      0.92       482
weighted avg       0.94      0.93      0.93       482



In [23]:
generate_classification_report(model, val_loader, class_names, device, name="Validation")


📊 Classification Report - Validation Set

              precision    recall  f1-score   support

     chicken       0.95      0.73      0.83        52
        duck       0.88      0.98      0.93       109

    accuracy                           0.90       161
   macro avg       0.92      0.86      0.88       161
weighted avg       0.91      0.90      0.90       161



In [24]:
generate_classification_report(model, train_loader, class_names, device, name="Train")


📊 Classification Report - Train Set

              precision    recall  f1-score   support

     chicken       0.99      1.00      0.99       275
        duck       1.00      0.99      1.00       622

    accuracy                           1.00       897
   macro avg       0.99      1.00      0.99       897
weighted avg       1.00      1.00      1.00       897

